# Capstone Project Notebook: Neighborhood locator expats

## Getting the notebook ready

In [20]:
#Import libraries
import pandas as pd
import requests
from bs4 import BeautifulSoup
import urllib.request, urllib.parse, urllib.error
print('Libraries imported')

Libraries imported


## Data collection and preparation

### Zip codes and geolocation data

To get the relevant zip-codes, I went to https://www.freemaptools.com/find-zip-codes-inside-radius.htm and searched for zip codes within 15 miles of 80021 (Westminster). The result is a list of zip-codes, which I have pasted in below. By using the len() function, I get that I have 87 zip codes within my radius. I then convert the list to a dataframe.

In [25]:
zipcodes = [80021,80005,80007,80038,80020,80027,80004,80031,80303,80003,80001,80006,80002,80035,80234,80030,80036,80260,80025,80305,80033,80023,80221,80026,80034,80212,80309,80402,80419,80614,80215,80241,80306,80307,80308,80310,80314,80233,80229,80214,80301,80211,80225,80037,80202,80024,80204,80226,80304,80265,80294,80293,80266,80216,80401,80205,80290,80264,80516,80602,80243,80244,80248,80250,80251,80256,80257,80259,80261,80263,80271,80273,80274,80281,80291,80299,80201,80217,80640,80228,80471,80203,80232,80218,80544,80403,80219]
print(len(zipcodes))
df_zip = pd.DataFrame(zipcodes, columns=['Zip'])
df_zip.head()

87


,Zip
0,80021
1,80005
2,80007
3,80038
4,80020


Data on citites, latitude and longitude in Colorado are obtained from https://public.opendatasoft.com/explore/dataset/us-zip-code-latitude-and-longitude/table/?refine.state=CO and downloaded as a csv file. By using the shape function, I can see that it has 680 zip codes. 

In [32]:
df_geo = pd.read_csv('zip-lat-and-lon.csv', sep=';')
df_geo.shape

(680, 8)

Then, I merge the two datasets by the zip code. As the shape is now 86 rows and 8 columns, all geospatial data got included, but there must have been one zip code in my list of zip codes, which was not included in the geospatial data. As only one was lost, I choose to proceed with the 86 zip codes without further investigation, but had several been lost, I would have had to look for additional data sources to help me get the rest of the geospatial data.

In [80]:
#Merging the two datasets by zip code
df = pd.merge(df_zip, df_geo, on='Zip')
df.shape

(86, 8)

In [81]:
#Let's just look at our data
df.head()

,Zip,City,State,Latitude,Longitude,Timezone,Daylight savings time flag,geopoint
0,80021,Broomfield,CO,39.881608,-105.099530,-7,1,"39.881608,-105.09953"
1,80005,Arvada,CO,39.843304,-105.118960,-7,1,"39.843304,-105.11896"
2,80007,Arvada,CO,39.833442,-105.185910,-7,1,"39.833442,-105.18591"
3,80038,Broomfield,CO,40.087835,-105.373507,-7,1,"40.087835,-105.373507"
4,80020,Broomfield,CO,39.934040,-105.054540,-7,1,"39.93404,-105.05454"


### Housing price information

Next step is to include rental information. The dataset of rental prices in Colorado provided by Zillow was found [here](https://public.opendatasoft.com/explore/dataset/rental-values/table/?disjunctive.regionname&disjunctive.city&disjunctive.state&disjunctive.metro&disjunctive.countyname&refine.state=CO) and is a much larger dataset of 22.431 rows and 9 columns.

In [33]:
df_rent = pd.read_csv('rental-values.csv', sep=';')
df_rent.shape

(22431, 9)

First, I will investigate the different house types ann price unit and the available data for each. Using the data per square meter seems reasonable, as the different sizes of the house units are then taking into account. However, as we would like a house with 1 bedroom, I'll also want to be able to compare zip codes on the rental price of a 1-bdr house. Therefore, I'll reduce the dataset to a dataset that returns the mean rental prices per sqm and the mean rental price of a 1-bdr house for each zipcode (where available). If you look closer at the data, you will realize that it has been collected over a range of years. While I'm mainly interested in new data, I don't need to predict the rental price, I just want to be able to compare. My data may be biased if the relative price between the zip codes have changed a lot in recent years, but for the purpose of this assignment, I will leave the rental value data as it is. 

In [42]:
print(df_rent['House type'].value_counts())
print(df_rent['Price unit'].value_counts())

House with 2 bedrooms              6939
Single Family Residential home.    6642
House with 1 bedroom               4869
House with 3 bedrooms              2152
Condo / Co-op                      1017
Studio                              656
Duplex / Triplex                    122
House with 4 bedrooms                34
Name: House type, dtype: int64
Per square meter    11291
Whole house         11140
Name: Price unit, dtype: int64


In [58]:
#First I do the data for the house with 1 bedroom. I only use data on the relevant house type 
df_1bdr = df_rent[df_rent['House type'] == 'House with 1 bedroom']
#Then I group them by zip code and return the mean rental value
df_1bdr = df_1bdr.groupby('ZIP Code')['Rental value'].mean()
#I generate a dataframe and rename the column
df_1bdr = pd.DataFrame(df_1bdr)
df_1bdr.reset_index(drop=False, inplace=True)
df_1bdr.rename(columns={'ZIP Code': 'Zip','Rental value': 'Rent 1bdr'}, inplace=True)
df_1bdr.head()

,Zip,Rent 1bdr
0,80002,667.364057
1,80010,541.887514
2,80011,684.534838
3,80012,532.109570
4,80013,560.979136


In [59]:
#Then I repeat the procedure, only for the price unit instead
df_sqm = df_rent[df_rent['Price unit'] == 'Per square meter']
#Then I group them by zip code and return the mean rental value
df_sqm = df_sqm.groupby('ZIP Code')['Rental value'].mean()
#I generate a dataframe and rename the column
df_sqm = pd.DataFrame(df_sqm)
df_sqm.reset_index(drop=False, inplace=True)
df_sqm.rename(columns={'ZIP Code': 'Zip','Rental value': 'Rent sqm'}, inplace=True)
df_sqm.head()

,Zip,Rent sqm
0,80002,1.918787
1,80003,1.571731
2,80010,1.537837
3,80011,1.628203
4,80012,1.489803


In [64]:
#Now let's add the two dataframes to our dataset using the left join, 
#(to avoid deleting any zip codes without rental price information available)
df2 = pd.merge(left=df,right=df_1bdr, how='left', left_on='Zip', right_on='Zip')
df2.shape
#We still got 86 zip codes, but with 9 columns instead of 8. Yay.

#Let's do the same for df_sqm
df2 = pd.merge(left=df2,right=df_sqm, how='left', left_on='Zip', right_on='Zip')
df2.shape
#Now we have 10 columns. Yay.

(86, 10)

As can be seen below, the rent data did not perform all that well, as it only returned rent data per sqm for 36 out of 87 zip codes. This means, that rental prices cannot be compared across all zip codes. 

In [67]:
#Let's see how well our rent data performed.
df2.isnull().sum()

Zip                            0
City                           0
State                          0
Latitude                       0
Longitude                      0
Timezone                       0
Daylight savings time flag     0
geopoint                       0
Rent 1bdr                     60
Rent sqm                      51
dtype: int64

It may be easier to get information on the median house value in a zip code instead, an use that as a proxy for rental prices. Zillow has also provided such a dataset available on OpenDataSoft, which I have sorted to include median home values per sqm in Colorado in 2019 (get data [here](https://public.opendatasoft.com/explore/dataset/unites-states-home-values/table/?q=CO&refine.state=CO&refine.date=2019)). This dataset provides data on a city level rather than on a zip code level, which means that it is less detailed, but should cover more of the areas we want to analyze for this study.

In [72]:
df_home = pd.read_csv('house-values.csv', sep=';')
df_home.shape
df_home.head()

(4908, 6)

In [77]:
#I will drop the irrelevant columns:
df_home.drop(columns=['State','Metro','County name','Date'], inplace=True)
#and rename the remaining columns
df_home.rename(columns={'Zip code': 'City','Home value ($ per m2)': 'House price sqm'}, inplace=True)

In [85]:
df_home.head()

,City,House price sqm
0,Ward,355
1,Yellow Jacket,166
2,Beulah,174
3,San Luis,80
4,Mosca,3568


In [97]:
#Let's get these data into our dataset and remove redundant columns
df2 = pd.merge(df,df_home, on='City')
df2 = df2.drop_duplicates()
df2.drop(columns=['State','Timezone','Daylight savings time flag'], inplace=True)
df2['Area']="("+ df2['Zip'].astype(str) + ") " + df2['City']
df2.head()

,Zip,City,Latitude,Longitude,geopoint,House price sqm,Area
0,80021,Broomfield,39.881608,-105.099530,"39.881608,-105.09953",269,(80021) Broomfield
12,80038,Broomfield,40.087835,-105.373507,"40.087835,-105.373507",269,(80038) Broomfield
24,80020,Broomfield,39.934040,-105.054540,"39.93404,-105.05454",269,(80020) Broomfield
36,80005,Arvada,39.843304,-105.118960,"39.843304,-105.11896",342,(80005) Arvada
48,80007,Arvada,39.833442,-105.185910,"39.833442,-105.18591",342,(80007) Arvada


Now we have 83 zip codes with geolocation and home price information. Good. We'll use this as the base for our Foursquare data retrieval. 

### Foursquare information

To get information about the different neighborhoods, I will use the foursquare API. I have chosen a radius of 1.5 km and limited the number of results to 200. 

In [93]:
#Preparing API credentials
CLIENT_ID = 'L4VLULQWHVKXWO4U0UY2CNYHAUHEQZ4YTHP0BDSQWZP0IZHQ' # your Foursquare ID
CLIENT_SECRET = 'IX0BS0UQ3MCNNN4D2A0LTSYWE2XCVBLXNY5NHDPXI0BAVRQT' # your Foursquare Secret
VERSION = '20200401' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: L4VLULQWHVKXWO4U0UY2CNYHAUHEQZ4YTHP0BDSQWZP0IZHQ
CLIENT_SECRET:IX0BS0UQ3MCNNN4D2A0LTSYWE2XCVBLXNY5NHDPXI0BAVRQT


In [119]:
#Defining function to get nearby venues in a neighborhood
def getNearbyVenues(names, latitudes, longitudes, radius=1500):
    LIMIT = 200
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Area', 
                  'Area Latitude', 
                  'Area Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [120]:
#Running the above function on all the areas in our dataset
dt_venues = getNearbyVenues(names=df2['Area'],
                                   latitudes=df2['Latitude'],
                                   longitudes=df2['Longitude']
                                  )

(80021) Broomfield
(80038) Broomfield
(80020) Broomfield
(80005) Arvada
(80007) Arvada
(80004) Arvada
(80003) Arvada
(80001) Arvada
(80006) Arvada
(80002) Arvada
(80027) Louisville
(80031) Westminster
(80035) Westminster
(80030) Westminster
(80036) Westminster
(80303) Boulder
(80305) Boulder
(80309) Boulder
(80306) Boulder
(80307) Boulder
(80308) Boulder
(80310) Boulder
(80314) Boulder
(80301) Boulder
(80304) Boulder
(80234) Denver
(80260) Denver
(80221) Denver
(80212) Denver
(80215) Denver
(80241) Denver
(80233) Denver
(80229) Denver
(80214) Denver
(80211) Denver
(80225) Denver
(80202) Denver
(80204) Denver
(80226) Denver
(80265) Denver
(80294) Denver
(80293) Denver
(80266) Denver
(80216) Denver
(80205) Denver
(80290) Denver
(80264) Denver
(80243) Denver
(80244) Denver
(80248) Denver
(80250) Denver
(80251) Denver
(80256) Denver
(80257) Denver
(80259) Denver
(80261) Denver
(80263) Denver
(80271) Denver
(80273) Denver
(80274) Denver
(80281) Denver
(80291) Denver
(80299) Denver
(80201) D

In [121]:
#Checking the results
print(dt_venues.shape)
dt_venues.head()

(3084, 7)


,Area,Area Latitude,Area Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,(80021) Broomfield,39.881608,-105.09953,Starbucks,39.880083,-105.096956,Coffee Shop
1,(80021) Broomfield,39.881608,-105.09953,LaMar's Donuts and Coffee,39.876984,-105.094064,Donut Shop
2,(80021) Broomfield,39.881608,-105.09953,Walnut Creek Liquor,39.880987,-105.092200,Liquor Store
3,(80021) Broomfield,39.881608,-105.09953,Walgreens,39.877497,-105.094410,Pharmacy
4,(80021) Broomfield,39.881608,-105.09953,King Soopers,39.876680,-105.097810,Grocery Store


Now I've got information on venues within my radius. Next step is to prepare the data to be used for an analysis. First, I'll make dummies for the different venue categories.

In [220]:
# one hot encoding
dt_onehot = pd.get_dummies(dt_venues[['Venue Category']], prefix="", prefix_sep="")

# Now, I add the area and lat/lon columns back to dataframe
dt_onehot['Area'] = dt_venues['Area'] 

#Right now, columns are in alphabetical order. Now we need to move the new column to the first position. 
cols = list(dt_onehot.columns.values)
cols.remove('Area')
cols.insert(0,'Area')
dt_onehot = dt_onehot.reindex(columns=cols)

dt_onehot.shape

(3084, 276)

Next, I group the data by area and sum up all the occurences of each venue category per area. It appears that only 73 out of the 83 zip codes has venues registered with them. This means, that my data set is now down to 73 areas. 

In [221]:
dt_grouped = dt_onehot.groupby('Area').sum().reset_index()
dt_grouped.shape

(73, 276)

I also wanted to be able to compare housing prices. Thus, I add the housing price information to the dataset.

In [222]:
df_house = df2[['Area','House price sqm']].copy()
df3 = pd.merge(dt_grouped,df_house, on='Area')
df3.head()

,Area,ATM,Accessories Store,Airport,Airport Terminal,Alternative Healer,American Restaurant,Arcade,Art Gallery,Art Museum,...,Vietnamese Restaurant,Warehouse Store,Water Park,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio,Zoo Exhibit,House price sqm
0,(80001) Arvada,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,342
1,(80002) Arvada,1,0,0,0,0,0,0,0,0,...,0,1,0,0,0,2,0,1,0,342
2,(80003) Arvada,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,342
3,(80004) Arvada,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,342
4,(80005) Arvada,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,342


## Data analysis

I will start with some exploratory data analysis. First I'll make a map with marks for all the areas in my study.

In [276]:
import folium

# create map of latitude and longitude values for Westminster, the center of our search radius. 
map_dt = folium.Map(location=[39.842285, -105.043716], zoom_start=10)

# add markers to map
for lat, lng, label in zip(df2['Latitude'], df2['Longitude'], df2['Zip']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='green',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.5,
        parse_html=False).add_to(map_dt)  
    
map_dt

I also want to know what kind of venue categories were found in my area and how many of each:

In [153]:
df_cat = pd.DataFrame(dt_grouped.sum(axis=0))
df_cat.drop(index='Area', inplace=True)
df_cat.sort_values(by=[0], ascending=False, inplace=True)
df_cat.reset_index(drop=False, inplace=True)
df_cat.to_csv(r'categories.csv', index = False)
df_cat

,index,0
0,Hotel,135
1,American Restaurant,119
2,Coffee Shop,119
3,Sandwich Place,113
4,Mexican Restaurant,107
...,...,...
270,Shoe Repair,1
271,Knitting Store,1
272,Sculpture Garden,1
273,Carpet Store,1


Seems like hotels, American restaurants, coffee shops and sandwich joints are to be found en masse. Not all that surprising. What I want next, is to be able to group these venues into my chosen set of preferences. Thus, I'll go through the list and look for keywords allowing me to group the variables.

In [184]:
import re

#Creating groups:
cafe = []
restaurant = []
bar = []
grocery = []
shopping = []
park = []
nature = []
sports = []

for ven in df_cat['index'].astype(str):
    if re.search('café|coffee|tea ',ven, re.IGNORECASE):
        if ven not in cafe:
            cafe.append(ven)
    if re.search('restaurant|pizza|food|mexican|steakhouse|burger|breakfast', ven, re.IGNORECASE):
        if ven not in restaurant:
            restaurant.append(ven)
    if re.search('brewery|beer|bar|pub', ven, re.IGNORECASE):
        if ven not in bar:
            bar.append(ven)
    if re.search('grocery|convenience', ven, re.IGNORECASE):
        if ven not in grocery:
            grocery.append(ven)
    if re.search('shop|store', ven, re.IGNORECASE):
        if ven not in shopping and ven not in cafe and ven not in grocery:
            shopping.append(ven)
    if re.search('park', ven, re.IGNORECASE):
        if ven not in park:
            park.append(ven)
    if re.search('nature|trail|mountain|river', ven, re.IGNORECASE):
        if ven not in nature:
            nature.append(ven)
            dogs.append(ven)
    if re.search('yoga|tennis|gym|bike', ven, re.IGNORECASE):
        if ven not in sports:
            sports.append(ven)
print(cafe)
print(restaurant)

['Coffee Shop', 'Café', 'Tea Room', 'Pet Café']
['American Restaurant', 'Mexican Restaurant', 'Pizza Place', 'Italian Restaurant', 'Fast Food Restaurant', 'Breakfast Spot', 'Burger Joint', 'Restaurant', 'Steakhouse', 'Asian Restaurant', 'New American Restaurant', 'Chinese Restaurant', 'Sushi Restaurant', 'Vegetarian / Vegan Restaurant', 'Vietnamese Restaurant', 'Seafood Restaurant', 'Thai Restaurant', 'Japanese Restaurant', 'Indian Restaurant', 'Mediterranean Restaurant', 'Food Truck', 'Brazilian Restaurant', 'Food', 'Greek Restaurant', 'Tex-Mex Restaurant', 'Food Court', 'German Restaurant', 'Korean Restaurant', 'Middle Eastern Restaurant', 'Cuban Restaurant', 'Caribbean Restaurant', 'Latin American Restaurant', 'Southern / Soul Food Restaurant', 'Bed & Breakfast', 'Tapas Restaurant', 'Ramen Restaurant', 'Cajun / Creole Restaurant', 'Food & Drink Shop', 'Dim Sum Restaurant', 'Hawaiian Restaurant', 'Food Service', 'French Restaurant']


Now, I want to group my Foursquare results based on the preference groups that I have just created. I will create new columns for all my preference groups and sum up how many venues in each group is to be found in each area.

In [223]:
df4 = df3

df4['cafe']=df4[cafe].sum(axis = 1, skipna = True) 
df4['restaurant']=df4[restaurant].sum(axis = 1, skipna = True) 
df4['bar']=df4[bar].sum(axis = 1, skipna = True) 
df4['grocery']=df4[grocery].sum(axis = 1, skipna = True) 
df4['shopping']=df4[shopping].sum(axis = 1, skipna = True) 
df4['park']=df4[park].sum(axis = 1, skipna = True) 
df4['nature']=df4[nature].sum(axis = 1, skipna = True) 
df4['sports']=df4[sports].sum(axis = 1, skipna = True) 

Next, I create a dataframe containing just the summary statistics. 

In [277]:
df5 = df4[['Area','House price sqm','cafe','restaurant','bar','grocery','shopping','park','nature','sports']].copy()
df5.to_csv(r'results.csv', index = False)
df5.head()

,Area,House price sqm,cafe,restaurant,bar,grocery,shopping,park,nature,sports
0,(80001) Arvada,342,0,0,0,0,0,0,0,0
1,(80002) Arvada,342,3,22,4,4,16,1,0,3
2,(80003) Arvada,342,1,13,2,2,5,1,0,2
3,(80004) Arvada,342,2,8,3,4,5,0,1,1
4,(80005) Arvada,342,0,2,0,1,1,0,1,1


Now, I want to start filtering my areas. First, I'll filter out any areas not containing at least one of *each* of my preferences. 

In [275]:
df6 = df5

df6.drop(df6.loc[df6['cafe']==0].index, inplace=True)
df6.drop(df6.loc[df6['restaurant']==0].index, inplace=True)
df6.drop(df6.loc[df6['bar']==0].index, inplace=True)
df6.drop(df6.loc[df6['grocery']==0].index, inplace=True)
df6.drop(df6.loc[df6['shopping']==0].index, inplace=True)
df6.drop(df6.loc[df6['park']==0].index, inplace=True)
df6.drop(df6.loc[df6['nature']==0].index, inplace=True)
df6.drop(df6.loc[df6['sports']==0].index, inplace=True)

df6

,Area,House price sqm,cafe,restaurant,bar,grocery,shopping,park,nature,sports
8,(80021) Broomfield,269,1,7,2,3,6,2,3,2
10,(80026) Lafayette,481,7,13,6,2,6,1,2,1
18,(80203) Denver,446,5,32,14,2,12,1,1,5
20,(80205) Denver,446,7,20,18,4,9,3,1,1
53,(80273) Denver,446,8,29,17,2,17,1,1,6
64,(80305) Boulder,778,4,7,3,2,8,2,2,3
66,(80401) Golden,725,2,18,2,2,14,1,1,2


As can be seen from the table above, I have narrowed down my search to seven different zip codes, locaed in Broomfield, Lafayette, Denver, Boulder and Golden. Cool. I want to see my proposed living locations:

In [229]:
#Merge dataset with dataset containing geolocations
df7 = pd.merge(df6,df2, on='Area')

In [234]:
# create map of latitude and longitude values for Westminster, the center of our search radius. 
map_dt = folium.Map(location=[39.842285, -105.043716], zoom_start=10)

# add markers to map
for lat, lng, label in zip(df7['Latitude'], df7['Longitude'], df7['Area']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='green',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.5,
        parse_html=False).add_to(map_dt)  
    
map_dt

Finally, let's check out the most common venue in each of the seven zip codes.

In [246]:
#Importing numpy library
import numpy as np

#Defining a function for putting the results into a dataframe
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

#Defining that I want the 10 most popular venues for each neighborhood
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Area']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Area'] = dt_grouped['Area']

for ind in np.arange(dt_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(dt_grouped.iloc[ind, :], num_top_venues)

#Merge dataset with relevant zip codes
df7A = pd.DataFrame(df7['Area'])
neighborhoods_venues_sorted = pd.merge(df7A,neighborhoods_venues_sorted, on='Area')

neighborhoods_venues_sorted

,Area,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,(80021) Broomfield,Trail,Home Service,Fast Food Restaurant,Pizza Place,Convenience Store,Shipping Store,Gym / Fitness Center,Sandwich Place,Park,Liquor Store
1,(80026) Lafayette,Coffee Shop,Mexican Restaurant,Pizza Place,Sandwich Place,Salon / Barbershop,Trail,Brewery,Mediterranean Restaurant,Bagel Shop,Department Store
2,(80203) Denver,Sandwich Place,Coffee Shop,Pizza Place,Yoga Studio,Mexican Restaurant,Italian Restaurant,Nightclub,American Restaurant,Marijuana Dispensary,Brewery
3,(80205) Denver,Bar,Brewery,Coffee Shop,Zoo Exhibit,Marijuana Dispensary,Liquor Store,Park,Burger Joint,Convenience Store,Pharmacy
4,(80273) Denver,Coffee Shop,American Restaurant,Brewery,Nightclub,Yoga Studio,Salon / Barbershop,Mexican Restaurant,Italian Restaurant,Marijuana Dispensary,Bar
5,(80305) Boulder,Bus Stop,Coffee Shop,Trail,Indian Restaurant,Café,Park,Pizza Place,Pub,Brewery,Lake
6,(80401) Golden,Pizza Place,Fast Food Restaurant,Mexican Restaurant,Pet Store,Pharmacy,Coffee Shop,Liquor Store,Sandwich Place,Hardware Store,Doctor's Office


This concludes the job of the data scientist. My husband and I are now left with a map of seven potential places to start searching for a new home, all fulfilling our preference criteria. We can use the tables to explore them in more detail, but ehether we want to weigh housing prices higher or lower than the amount of coffee shops and nature areas are up to us. From the table on most common venues, we can for example learn that the most common venue in Boulder i a bus stop, indicating good public transport. Also, Boulder has an abundance of coffee shops and trails, which I really love. Denver provides better opportunities in terms of dining and going out, while Broomfield has trails, home service and fast food as top three, indicating that 'city-life' with restaurants and cute coffee shops may be harder to find in Broomfield. Golden has a lot of eating options. It does surprise me that Golden does not have trails and nature more prominently, as it is located right next to the mountains. Oh, well, Foursquare cannot know everything about the world... yet, at least ;)